## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kodiak,US,57.7900,-152.4072,47.71,71,0,9.22,clear sky
1,1,Airai,TL,-8.9266,125.4092,73.26,54,23,4.16,few clouds
2,2,Ribeira Grande,PT,38.5167,-28.7000,59.20,67,100,10.67,overcast clouds
3,3,Laguna,US,38.4210,-121.4238,92.14,24,0,9.22,clear sky
4,4,Albany,US,42.6001,-73.9662,56.17,63,13,11.56,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria  
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 25
What is your desired maximum temperature for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kodiak,US,57.7900,-152.4072,47.71,71,0,9.22,clear sky
1,1,Airai,TL,-8.9266,125.4092,73.26,54,23,4.16,few clouds
2,2,Ribeira Grande,PT,38.5167,-28.7000,59.20,67,100,10.67,overcast clouds
4,4,Albany,US,42.6001,-73.9662,56.17,63,13,11.56,few clouds
6,6,Pleshanovo,RU,52.8333,53.4833,45.27,91,6,5.23,clear sky
7,7,Narsaq,GL,60.9167,-46.0500,48.42,83,95,1.88,overcast clouds
9,9,Vila Franca Do Campo,PT,37.7167,-25.4333,59.41,84,91,6.87,overcast clouds
10,10,Mataura,NZ,-46.1927,168.8643,61.48,60,85,5.32,light rain
14,14,Port Alfred,ZA,-33.5906,26.8910,69.84,46,0,10.38,clear sky
17,17,Ushuaia,AR,-54.8000,-68.3000,46.06,66,0,2.30,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kodiak,US,57.7900,-152.4072,47.71,71,0,9.22,clear sky
1,1,Airai,TL,-8.9266,125.4092,73.26,54,23,4.16,few clouds
2,2,Ribeira Grande,PT,38.5167,-28.7000,59.20,67,100,10.67,overcast clouds
4,4,Albany,US,42.6001,-73.9662,56.17,63,13,11.56,few clouds
6,6,Pleshanovo,RU,52.8333,53.4833,45.27,91,6,5.23,clear sky
...,...,...,...,...,...,...,...,...,...,...
681,681,Launceston,AU,-41.4500,147.1667,54.81,65,83,18.41,light rain
682,682,Coahuayana,MX,18.7333,-103.6833,74.97,60,16,2.48,few clouds
685,685,Sindor,RU,62.8629,51.8905,33.51,80,24,7.78,few clouds
686,686,Canatlan,MX,24.5167,-104.7833,63.48,62,100,4.81,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kodiak,US,47.71,clear sky,57.7900,-152.4072,
1,Airai,TL,73.26,few clouds,-8.9266,125.4092,
2,Ribeira Grande,PT,59.20,overcast clouds,38.5167,-28.7000,
4,Albany,US,56.17,few clouds,42.6001,-73.9662,
6,Pleshanovo,RU,45.27,clear sky,52.8333,53.4833,
7,Narsaq,GL,48.42,overcast clouds,60.9167,-46.0500,
9,Vila Franca Do Campo,PT,59.41,overcast clouds,37.7167,-25.4333,
10,Mataura,NZ,61.48,light rain,-46.1927,168.8643,
14,Port Alfred,ZA,69.84,clear sky,-33.5906,26.8910,
17,Ushuaia,AR,46.06,clear sky,-54.8000,-68.3000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")
        

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Sk

In [9]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kodiak,US,47.71,clear sky,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center
2,Ribeira Grande,PT,59.20,overcast clouds,38.5167,-28.7000,Quinta da Meia Eira
6,Pleshanovo,RU,45.27,clear sky,52.8333,53.4833,Gostevoy Dom Magiya Uyuta
7,Narsaq,GL,48.42,overcast clouds,60.9167,-46.0500,Sleep In Narsaq
9,Vila Franca Do Campo,PT,59.41,overcast clouds,37.7167,-25.4333,Pestana Bahia Praia
...,...,...,...,...,...,...,...
679,Tromso,NO,37.42,overcast clouds,69.6496,18.9570,"Radisson Blu Hotel, Tromsø"
681,Launceston,AU,54.81,light rain,-41.4500,147.1667,Hotel Grand Chancellor Launceston
682,Coahuayana,MX,74.97,few clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
686,Canatlan,MX,63.48,overcast clouds,24.5167,-104.7833,AUTOLAVADO El Negro


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))